## Transformer-based Natural Language Generator

This notebook present the implementation of a Transformer-based language generator meant to be used with data generated through the M2M framework. The model takes as an input the previous turn of the conversation and a pre-processed sequence of dialogue acts entailing the semantic content of the reply. The proposed architecture includes two separate Encoders for user's previous turn and the sequence of dialogue acts to be translated.
The output is a natural language utterance that convey the semantic content corried by the dialogue acts.

#### Imports

In [38]:
from pymongo import MongoClient

import string
import math
import re
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn.utils.rnn import pad_sequence

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 20

In [41]:
skip_training = False

#### Loading data

In [42]:
# set to true to download fetch data from db
download = True

In [43]:
#db_name = 'YOUR_DB_NAME'
#collection_name = 'YOUR_COLLECTION NAME'
db_name = 'thesis'
collection_name = 'sirto3'

In [44]:
if download:
    client = MongoClient('localhost', port=27017)
    db = client[db_name]
    dialogues = [dialogue for dialogue in db[collection_name].find({})]
else:
    # @TODO
    # load data from local repository
    pass

#### Different representations of the data
- each element "slot" "value" "act" "value" is treated as a single word of a sentence
- seq of annotation is directly converted into a string

In [45]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

#### Process string to lowercase and strip of punctuation

In [46]:
def normalizeString(s):
    s = s.lower().strip()
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

#### Read dialogues into pairs of <annotations, rewrite>

In [47]:
def readDialogues(dialogues, lang1, lang2):
    pairs = []
    for dialogue in dialogues:
        for i in range(0, len(dialogue['annotatedMessages']), 2):
            if i+1 >= len(dialogue['annotatedMessages']):
                continue
            prev_user_turn = dialogue['annotatedMessages'][i]
            system_turn = dialogue['annotatedMessages'][i+1]
            if prev_user_turn['rewrite'] != '' and system_turn['rewrite'] != '':
                if prev_user_turn['sender'] == 'User' and system_turn['sender'] == 'Chatbot':
                    annotations = ''
                    for annotation in system_turn['annotations']:
                        annotations += normalizeString(str(annotation))
                    system_rewrite = normalizeString(system_turn['rewrite'])
                    user_prev_rewrite = normalizeString(prev_user_turn['rewrite'])
                    pairs.append([annotations, system_rewrite, user_prev_rewrite])
                else:
                    continue
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    return input_lang, output_lang, pairs

#### Filter out excessively long turns

In [48]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        len(p[2].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [49]:
def prepareData(dialogues, lang1, lang2):
    input_lang, output_lang, pairs = readDialogues(dialogues, lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        input_lang.addSentence(pair[2])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

In [50]:
input_lang, output_lang, pairs = prepareData(dialogues, 'annotations', 'line')
print(random.choice(pairs))

Read 287 sentence pairs
Trimmed to 216 sentence pairs
Counting words...
Counted words:
annotations 309
line 273
[' act negate slot none value none  act request slot license plate value none ', 'oops there is something wrong with the licence plate format could you please write that again', 'sure my licence plate code is jdr and i live on sj str msv gen']


In [51]:
pairs[470:475]

[]

#### Transform pairs of sentences into pairs of tensor of indices

In [52]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    user_tensor = tensorFromSentence(input_lang, pair[2])
    return (input_tensor, target_tensor, user_tensor)

In [53]:
tensor_pairs = [tensorsFromPair(pair) for pair in pairs]

### Custom dataset

In [54]:
class SiirtoDialogueDataset(Dataset):

    def __init__(self, pairs, transform=None):
        
        self.pairs = pairs
        self.transform = transform

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        sample = (self.pairs[idx][0], self.pairs[idx][1], self.pairs[idx][2])

        if self.transform:
            sample = self.transform(sample)

        return sample

In [55]:
PADDING_VALUE = 0

def collate(list_of_samples):

    max_src_seq_length = max([len(src_seq) for (src_seq, tgt_seq, usr_seq) in list_of_samples])
    max_tgt_seq_length = max([len(tgt_seq) for (src_seq, tgt_seq, usr_seq) in list_of_samples])
    max_usr_seq_length = max([len(usr_seq) for (src_seq, tgt_seq, usr_seq) in list_of_samples])
    batch_size = len(list_of_samples)
    
    src_seqs = torch.zeros([max_src_seq_length, batch_size], dtype=torch.long)
    src_mask = torch.full([max_src_seq_length, batch_size], True, dtype=torch.bool)
    tgt_seqs = torch.zeros([max_tgt_seq_length+1, batch_size], dtype=torch.long) 
    usr_seqs = torch.zeros([max_usr_seq_length, batch_size], dtype=torch.long)
    usr_mask = torch.full([max_usr_seq_length, batch_size], True, dtype=torch.bool)
        
    for i, (src_seq, tgt_seq, usr_seq) in enumerate(list_of_samples):
        src_seqs[:len(src_seq),i] = src_seq
        src_mask[:len(src_seq),i] = False
        tgt_seqs[1:len(tgt_seq)+1,i] = tgt_seq  
        usr_seqs[:len(usr_seq),i] = usr_seq
        usr_mask[:len(usr_seq),i] = False
    
    return src_seqs, src_mask, tgt_seqs, usr_seqs, usr_mask

In [56]:
# select test pairs
test_pairs = []
for _ in range(100):
    test_pairs.append(tensor_pairs.pop(random.randint(0, len(tensor_pairs))))

In [57]:
trainset = SiirtoDialogueDataset(tensor_pairs)

In [58]:
trainloader = DataLoader(dataset=trainset, batch_size=16, shuffle=True, collate_fn=collate, pin_memory=True)

In [59]:
testset = SiirtoDialogueDataset(test_pairs)

# Transformer

In [60]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        assert (d_model % 2) == 0
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class NoamOptimizer:
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
    
    def zero_grad(self):
        self.optimizer.zero_grad()

In [61]:
class EncoderBlock(nn.Module):
    def __init__(self, n_features, n_heads, n_hidden=64, dropout=0.1):

        super(EncoderBlock, self).__init__()
        
        self.att = nn.MultiheadAttention(n_features, n_heads)
        self.norm1 = nn.LayerNorm(n_features)
        self.norm2 = nn.LayerNorm(n_features)
        self.dropout2 = nn.Dropout(p=dropout)
        self.dropout3 = nn.Dropout(p=dropout)
        
        # feedforward
        self.linear1 = nn.Linear(n_features, n_hidden)
        self.dropout1 = nn.Dropout(p=dropout)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(n_hidden, n_features)

    def forward(self, x, mask):

        out, attn_output_weights = self.att(x, x, x, key_padding_mask=mask)
        
        # skip
        out = self.dropout2(out)
        out += x
        out = self.norm1(out)

        skip = out
        
        # feedforward
        out = self.linear1(out)
        out = self.dropout1(out)
        out = self.relu1(out)
        out = self.linear2(out)        
                
        # skip
        out = self.dropout3(out)
        out += skip
        out = self.norm2(out)
        
        return out

In [62]:
class Encoder(nn.Module):
    def __init__(self, src_vocab_size, n_blocks, n_features, n_heads, n_hidden=64, dropout=0.1):

        super(Encoder, self).__init__()
        
        self.embedding = nn.Embedding(src_vocab_size, n_features)
        self.pe = PositionalEncoding(n_features, dropout, max_len=MAX_LENGTH)
        self.encoder_blocks = nn.ModuleList([EncoderBlock(n_features, n_heads, n_hidden, dropout) for _ in range(n_blocks)])

    def forward(self, x, mask):
        
        out = self.embedding(x)  
        out = self.pe(out)
        
        for encoder_block in self.encoder_blocks:
            out = encoder_block(out, mask)
        
        return out

In [63]:
def subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1).float()
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [64]:
class DecoderBlock(nn.Module):
    def __init__(self, n_features, n_heads, n_hidden=64, dropout=0.1):

        super(DecoderBlock, self).__init__()
        
        self.masked_att = nn.MultiheadAttention(n_features, n_heads)
        self.att = nn.MultiheadAttention(n_features, n_heads)
        
        self.norm1 = nn.LayerNorm(n_features)
        self.norm2 = nn.LayerNorm(n_features)
        self.norm3 = nn.LayerNorm(n_features)
        
        self.dropout1 = nn.Dropout(p=dropout)
        self.dropout2 = nn.Dropout(p=dropout)
        self.dropout3 = nn.Dropout(p=dropout)
        self.dropout4 = nn.Dropout(p=dropout)
        
        # MLP
        self.linear1 = nn.Linear(n_features, n_hidden)
        self.linear2 = nn.Linear(n_hidden, n_features)
        self.relu1 = nn.ReLU()

    def forward(self, y, z, src_mask, tgt_mask):
        
        out, masked_attn_output_weights = self.masked_att(y, y, y, attn_mask=tgt_mask)
        
        #skip
        out = self.dropout1(out)
        out = out + y
        out = self.norm1(out)
        
        skip = out
        
        out, attn_output_weights = self.att(out, z, z, key_padding_mask=src_mask)
        
        #skip
        out = self.dropout2(out)
        out = out + skip
        out = self.norm2(out)
        
        skip = out  
        
        # feedforward
        out = self.linear1(out)
        out = self.dropout3(out)
        out = self.relu1(out)
        out = self.linear2(out) 
        
                
        #skip
        out = self.dropout4(out)
        out = out + skip
        out = self.norm3(out)
        
        return out

In [65]:
class Decoder(nn.Module):
    def __init__(self, tgt_vocab_size, n_blocks, n_features, n_heads, n_hidden=64, dropout=0.1):

        super(Decoder, self).__init__()
        
        self.embedding = nn.Embedding(tgt_vocab_size, n_features)
        self.pe = PositionalEncoding(n_features, dropout, max_len=MAX_LENGTH)
        
        self.decoder_blocks = nn.ModuleList([DecoderBlock(n_features, n_heads, n_hidden, dropout) for _ in range(n_blocks)])
        
        self.linear = nn.Linear(n_features, tgt_vocab_size)
        self.log_soft = nn.LogSoftmax(dim=2)
        
    def forward(self, y, z, src_mask):

        tgt_mask = subsequent_mask(y.size(0)) 
        
        out = self.embedding(y)
        out = self.pe(out)
        
        for decoder_block in self.decoder_blocks:
            out = decoder_block(out, z, src_mask, tgt_mask)
        
        out = self.linear(out)
        out = self.log_soft(out)
        
        return out

## Training

In [70]:
import torch.nn as nn

# Create the transformer model
n_features = 256
encoder = Encoder(src_vocab_size=input_lang.n_words, n_blocks=3, n_features=n_features,
                  n_heads=16, n_hidden=1024)
user_encoder = Encoder(src_vocab_size=input_lang.n_words, n_blocks=3, n_features=n_features,
                  n_heads=16, n_hidden=256)
decoder = Decoder(tgt_vocab_size=output_lang.n_words, n_blocks=3, n_features=n_features,
                  n_heads=16, n_hidden=1024)

encoder.to(device)
user_encoder.to(device)
decoder.to(device)

Decoder(
  (embedding): Embedding(273, 256)
  (pe): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (decoder_blocks): ModuleList(
    (0): DecoderBlock(
      (masked_att): MultiheadAttention(
        (out_proj): Linear(in_features=256, out_features=256, bias=True)
      )
      (att): MultiheadAttention(
        (out_proj): Linear(in_features=256, out_features=256, bias=True)
      )
      (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (norm3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (dropout3): Dropout(p=0.1, inplace=False)
      (dropout4): Dropout(p=0.1, inplace=False)
      (linear1): Linear(in_features=256, out_features=1024, bias=True)
      (linear2): Linear(in_features=1024, out_features=256, bias=True)
      (relu1): ReLU()
    )
    (1): DecoderBloc

In [71]:
parameters = list(encoder.parameters()) + list(user_encoder.parameters()) + list(decoder.parameters())
adam = torch.optim.Adam(parameters, lr=0, betas=(0.9, 0.98), eps=1e-9)
optimizer = NoamOptimizer(n_features, 2, 10000, adam)

In [72]:
trainloader = DataLoader(dataset=trainset, batch_size=64, shuffle=True, collate_fn=collate, pin_memory=True)

In [73]:
if not skip_training:
    
    n_epochs = 80
    iter = 0
    
    for epoch in range(n_epochs):
        for (src_seqs, src_mask, tgt_seqs, usr_seqs, usr_mask) in trainloader:
            
            # transpose masks
            src_mask = src_mask.T
            usr_mask = usr_mask.T
            
            # reset gradients
            optimizer.zero_grad()
            
            encoded = encoder(src_seqs, src_mask)
            user_encoded = user_encoder(usr_seqs, usr_mask)

            concat = torch.cat([encoded, user_encoded], dim=0)
            concat_mask = torch.cat([src_mask, usr_mask], dim=1)
                 
            pred_seqs = decoder(tgt_seqs[:-1], concat, src_mask=concat_mask)
            
            loss = 0
            
            for (pred, targ) in zip(pred_seqs, tgt_seqs[1:]):
                loss += F.nll_loss(pred, targ, ignore_index=0)
            
            loss.backward()
            
            optimizer.step()
            
            iter += 1
            
        print('Epoch: {}, Iter: {}, Loss: {}'.format(epoch, iter, loss.item()))

Epoch: 0, Iter: 2, Loss: 107.34777069091797
Epoch: 1, Iter: 4, Loss: 107.15493774414062
Epoch: 2, Iter: 6, Loss: 107.4543685913086
Epoch: 3, Iter: 8, Loss: 97.85518646240234
Epoch: 4, Iter: 10, Loss: 97.08071899414062
Epoch: 5, Iter: 12, Loss: 91.45077514648438
Epoch: 6, Iter: 14, Loss: 107.62029266357422


KeyboardInterrupt: 

In [625]:
def save_model(model, filename):
    try:
        do_save = input('Do you want to save the model (type yes to confirm)? ').lower()
        if do_save == 'yes':
            torch.save(model.state_dict(), filename)
            print('Model saved to %s.' % (filename))
        else:
            print('Model not saved.')
    except:
        raise Exception()


def load_model(model, filename, device):
    model.load_state_dict(torch.load(filename, map_location=lambda storage, loc: storage))
    print('Model loaded from %s.' % filename)
    model.to(device)
    model.eval()

In [626]:
if not skip_training:
    save_model(encoder, 'user_tr_encoder_3105.pth')
    save_model(user_encoder, 'user_user_tr_encoder_3105.pth')
    save_model(decoder, 'user_tr_decoder_3105.pth')
else:
    encoder = Encoder(src_vocab_size=trainset.input_lang.n_words, n_blocks=3, n_features=256, n_heads=16, n_hidden=1024)
    load_model(encoder, 'user_tr_encoder.pth', device)
    
    user_encoder = Encoder(src_vocab_size=trainset.input_lang.n_words, n_blocks=3, n_features=256, n_heads=16, n_hidden=1024)
    load_model(user_encoder, 'user_user_tr_encoder.pth', device)
    
    decoder = Decoder(tgt_vocab_size=trainset.output_lang.n_words, n_blocks=3, n_features=256, n_heads=16, n_hidden=1024)
    load_model(decoder, 'user_tr_decoder.pth', device)

Do you want to save the model (type yes to confirm)? yes
Model saved to user_tr_encoder_3105.pth.
Do you want to save the model (type yes to confirm)? yes
Model saved to user_user_tr_encoder_3105.pth.
Do you want to save the model (type yes to confirm)? yes
Model saved to user_tr_decoder_3105.pth.


## Evaluation

In [627]:
def translate(encoder, decoder, user_encoder, src_seq, usr_seq):

    src_seq = src_seq.view(len(src_seq), -1)
    src_mask = torch.full(src_seq.shape, False, dtype=torch.bool).T
    tgt_seq = torch.full((1,1), SOS_token, dtype=torch.long)
    
    usr_seq = usr_seq.view(len(usr_seq), -1)
    usr_mask = torch.full(usr_seq.shape, False, dtype=torch.bool).T
    
    encoder.eval()
    user_encoder.eval()
    decoder.eval()
    
    z = encoder(src_seq, src_mask)
    u = user_encoder(usr_seq, usr_mask)
    
    concat = torch.cat([z, u], dim=0)
    concat_mask = torch.cat([src_mask, usr_mask], dim=1)
    
    while tgt_seq[-1,:] != EOS_token:
        output = decoder(tgt_seq, concat, concat_mask)
        _, pred_seq = torch.max(output, dim=2)
        tgt_seq = torch.cat([tgt_seq, pred_seq[-1,:].view(-1,1)], dim=0)      
        
    return pred_seq

In [628]:
print('Translate training data:')
print('-----------------------------')
for i in range(5):
    src_sentence, tgt_sentence, usr_sentence = trainset[np.random.choice(len(trainset))]
    print('Annotation >', ' '.join(input_lang.index2word[i.item()] for i in src_sentence))
    print('Previous user turn >', ' '.join(input_lang.index2word[i.item()] for i in usr_sentence))
    print('=', ' '.join(output_lang.index2word[i.item()] for i in tgt_sentence))
    out_sentence = translate(encoder, decoder, user_encoder, src_sentence, usr_sentence)
    print('<', ' '.join(output_lang.index2word[i.item()] for i in out_sentence), '\n')

Translate training data:
-----------------------------
Annotation >  act request slot license plate value none  EOS
Previous user turn > accept EOS
= please provide license plate number EOS
< please provide license plate number EOS 

Annotation >  act request slot license plate value none  EOS
Previous user turn > yes it is . EOS
= okay . please provide license plate number . EOS
< please provide license plate number . EOS 

Annotation >  act request slot terms and conditions value none  EOS
Previous user turn > my area is linnavuorentie EOS
= please accept terms and conditions EOS
< please accept the terms and conditions EOS 

Annotation >  act notify success slot none value none  act inform slot registration value true  EOS
Previous user turn >   EOS
= thank you the registration is completed EOS
< thank you you have now registered EOS 

Annotation >  act request slot phone number value none  EOS
Previous user turn > it is accepted . EOS
= please provide phone number for reference . E

In [629]:
print('Translate test data:')
print('-----------------------------')
for i in range(len(5)):
    input_sentence, target_sentence, user_sentence = testset[np.random.choice(len(testset))]
    print('Annotations >', ' '.join(input_lang.index2word[i.item()] for i in input_sentence))
    print('Previpus user turn >', ' '.join(input_lang.index2word[i.item()] for i in user_sentence))
    print('=', ' '.join(output_lang.index2word[i.item()] for i in target_sentence))
    output_sentence = translate(encoder, decoder, user_encoder, input_sentence, user_sentence)
    print('<', ' '.join(output_lang.index2word[i.item()] for i in output_sentence), '\n')

Translate test data:
-----------------------------
Annotations >  act greeting slot none value none  act propose slot registration to siirtosoitto value none  EOS
Previpus user turn > hai there EOS
= need to register to siirtosoitto EOS
< hello would you like to register to siirtosoitto ? EOS 

Annotations >  act greeting slot none value none  act propose slot registration to siirtosoitto value none  EOS
Previpus user turn > good morning ! EOS
= good morning . would you want to registrate to siirtosoitto ? EOS
< hello would you like to register to siirtosoitto ? EOS 

Annotations >  act greeting slot none value none  act propose slot registration to siirtosoitto value none  EOS
Previpus user turn > hello ! EOS
= hello would you like to register to this site ? EOS
< hello would you like to register to siirtosoitto ? EOS 

Annotations >  act greeting slot none value none  act propose slot registration to siirtosoitto value none  EOS
Previpus user turn > hi EOS
= hi can i register to siir

< please provide your pincode EOS 

Annotations >  act request slot license plate value none  EOS
Previpus user turn > crusellinsitta is the notification area EOS
= give me your license plate number EOS
< please provide license plate number EOS 

Annotations >  act notify success slot none value none  act inform slot registration value true  EOS
Previpus user turn > my pin is . my area is kvirinne and i accept the terms and conditions . EOS
= thank you your registration has been confirmed . EOS
< thank you . your registration is completed . EOS 

Annotations >  act request slot phone number value none  EOS
Previpus user turn > yes EOS
= add your phone number EOS
< please provide phone number EOS 

Annotations >  act confirm slot area value henriksberginkuja  EOS
Previpus user turn > license plate is iat notification area is henrimsberginkuma EOS
= area is henriksberginkuja ? correct ? EOS
< is the area is the area is the area is the area EOS 

Annotations >  act request slot license pl

< hello would you like to register to siirtosoitto EOS 

Annotations >  act request slot area value none  EOS
Previpus user turn > license plate numbers are bnf and cop . EOS
= what area are you interested in to get notification ? EOS
< please provide your pincode . EOS 

Annotations >  act request slot pincode value none  EOS
Previpus user turn > the number is  EOS
= what is the pincode ? EOS
< please provide your pincode EOS 

Annotations >  act request slot terms and conditions value none  EOS
Previpus user turn > it is gcp . EOS
= please accept the terms and conditions . EOS
< please accept the terms and conditions . EOS 

Annotations >  act greeting slot none value none  act propose slot registration to siirtosoitto value none  EOS
Previpus user turn > hi chatbot EOS
= would you like to get registered to this new siirto service EOS
< hello would you like to register to siirtosoitto EOS 

Annotations >  act notify success slot none value none  act inform slot registration value tru